# TensorRT

Di notebook ini, kita akan menggunakan TensorRT untuk mengoptimalkan model PyTorch untuk inferensi. Kami akan melatih model CNN sederhana pada kumpulan data MNIST, mengonversinya menjadi mesin TensorRT menggunakan ONNX, lalu melakukan inferensi menggunakan model mesin TensorRT yang dioptimalkan dan mengevaluasi ukuran dan keakuratan model. Notebook ini memerlukan GPU NVIDIA dengan dukungan CUDA atau perangkat NVIDIA Jetson.

## Siapkan TensorRT

Pertama, instal tensorrt dan torch menggunakan pip dan impor modul yang diperlukan

In [1]:
%pip install torch torchvision
%pip install tensorrt==8.6.1
%pip install pycuda onnx onnxruntime
%pip install --no-cache-dir --extra-index-url https://pypi.nvidia.com pytorch-quantization==2.1.2

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1-py2.py3-none-any.whl size=16972 sha256=cc31e3c094d7965b747075a7e397e004d74cbf5b6e78370d04f2ce3f6d7577d8
  Stored in directory: /root/.cache/pip/wheels/6d/29/56/abdffd4c604f255b5254bef3f1c598ab7811ea020540599438
Successfully built tensorrt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.4/92.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.quantization
import pathlib
import numpy as np
import torch.onnx
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import onnx
import onnxruntime

from pytorch_quantization import nn as quant_nn
from pytorch_quantization import quant_modules
from pytorch_quantization import calib
from tqdm import tqdm

## Latih Model PyTorch dan Ekspor ke ONNX

Selanjutnya, latih model CNN sederhana pada kumpulan data MNIST dan ekspor ke format ONNX

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

train_dataset = datasets.MNIST('./data', train=True, download=True,transform=transform)
test_dataset = datasets.MNIST('./data', train=False,transform=transform)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(12 * 13 * 13, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        return output


train_loader = torch.utils.data.DataLoader(train_dataset, 32)
test_loader = torch.utils.data.DataLoader(test_dataset, 32)

device = "cpu"

epochs = 1

model = Net().to(device)
optimizer = optim.Adam(model.parameters())

model.train()

for epoch in range(1, epochs+1):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

MODEL_DIR = pathlib.Path("./models")
MODEL_DIR.mkdir(exist_ok=True)
torch.save(model.state_dict(), MODEL_DIR / "original_model.p")

x, _ = next(iter(train_loader))
torch.onnx.export(model,
                  x,
                  MODEL_DIR / "mnist_model.onnx",
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes={'input' : {0 : 'batch_size'},
                                'output' : {0 : 'batch_size'}})

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.5MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 514kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.84MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 10.3MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.428713
Train Epoch: 1 [32/60000 (0%)]	Loss: 2.462280
Train Epoch: 1 [64/60000 (0%)]	Loss: 2.208610
Train Epoch: 1 [96/60000 (0%)]	Loss: 2.027727
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.094260
Train Epoch: 1 [160/60000 (0%)]	Loss: 2.110198
Train Epoch: 1 [192/60000 (0%)]	Loss: 1.657317
Train Epoch: 1 [224/60000 (0%)]	Loss: 1.784680
Train Epoch: 1 [256/60000 (0%)]	Loss: 1.808799
Train Epoch: 1 [288/60000 (0%)]	Loss: 1.449356
Train Epoch: 1 [320/60000 (1%)]	Loss: 1.527941
Train Epoch: 1 [352/60000 (1%)]	Loss: 1.307230
Train Epoch: 1 [384/60000 (1%)]	Loss: 1.368755
Train Epoch: 1 [416/60000 (1%)]	Loss: 1.246383
Train Epoch: 1 [448/60000 (1%)]	Loss: 1.105210
Train Epoch: 1 [480/60000 (1%)]	Loss: 1.494388
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.288670
Train Epoch: 1 [544/60000 (1%)]	Loss: 0.987826
Train Epoch: 1 [576/60000 (1%)]	Loss: 1.250500
Train Epoch: 1 [608/60000 (1%)]	Loss:

## Konversikan Model ONNX ke TensorRT

Untuk mengonversi model ONNX ke mesin TensorRT menggunakan TensorRT Python API. Pertama, inisialisasi komponen TensorRT yaitu logger, builder, dan jaringan. Selanjutnya, tentukan parser ONNX untuk mengurai model ONNX dari file ONNX ke jaringan TensorRT. Kemudian, buat konfigurasi pembangun untuk menetapkan parameter pembangunan dan batas kumpulan memori untuk ruang kerja di TensorRT. Kemudian, buat profil pengoptimalan untuk menangani bentuk masukan dinamis dengan ukuran batch 32, ukuran saluran 1, dan dimensi gambar 28x28. Selanjutnya, buat dan buat serial mesin TensorRT menggunakan jaringan dan pembuat yang dikonfigurasi, lalu simpan ke disk. Terakhir, skrip dibersihkan dengan menghapus pembuat dan objek jaringan untuk mengosongkan sumber daya.

In [4]:
onnx_path = MODEL_DIR / "mnist_model.onnx"
trt_path = MODEL_DIR / 'mnist_engine_pytorch.trt'

# initialize TensorRT engine and parse ONNX model
logger = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(logger)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

parser = trt.OnnxParser(network, logger)
parser.parse_from_file(str(onnx_path))

# set up the builder config and coptimization profile
config = builder.create_builder_config()
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)

profile = builder.create_optimization_profile()
profile.set_shape("input", (32, 1, 28, 28), (32, 1, 28, 28), (32, 1, 28, 28))
config.add_optimization_profile(profile)

# serialize the engine, then save to disk
serialized_engine = builder.build_serialized_network(network, config)
with open(str(trt_path), 'wb') as f:
    f.write(serialized_engine)

# free up resources
del builder
del network

## Jalankan Inferensi dan Periksa Akurasi

Terakhir, jalankan inferensi lalu bandingkan akurasi model mesin TensorRT dengan model ONNX pada set data pengujian.

Untuk menjalankan pengujian model ONNX, muat model dan uji integritas model model lalu ulangi Data Loader yang diberikan. Untuk setiap batch, konversikan data input ke array NumPy dan masukkan ke dalam sesi ONNX Runtime. Sekali, diperoleh keluaran yang diubah kembali menjadi tensor PyTorch. Kemudian, hitung akumulasi kerugian kemungkinan log negatif
dan jumlah prediksi yang benar untuk mengukur keakuratan model.

Untuk menguji model tensorRT, pertama-tama, muat mesin serial dari disk, dan inisialisasi runtime TensorRT. Kemudian, deserialisasi mesin dan buat konteks eksekusi dibuat. Selanjutnya, alokasikan memori untuk data input dan output pada GPU, atur binding untuk eksekusi TensorRT, dan buat aliran CUDA untuk mengelola transfer data asinkron antara CPU dan GPU. Kemudian, Ulangi Data Loader yang diberikan dan untuk setiap batch, konversikan data input ke array NumPy dan transfer ke GPU, sebelum mengeksekusi model secara asinkron, lalu transfer prediksi kembali ke CPU. Jalankan sinkronisasi untuk memastikan koordinasi yang tepat antar thread. Selanjutnya, bentuk ulang keluaran dan konversikan ke tensor PyTorch untuk menghitung akumulasi kerugian kemungkinan log negatif dan jumlah prediksi yang benar untuk mengukur keakuratan model. Terakhir, kosongkan memori dan sumber daya CUDA

In [5]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

def test_onnx(model_name, data_loader):
    onnx_model = onnx.load(model_name)
    onnx.checker.check_model(onnx_model)
    ort_session = onnxruntime.InferenceSession(model_name)
    test_loss = 0
    correct = 0
    for data, target in data_loader:
        ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(data)}
        output = ort_session.run(None, ort_inputs)[0]
        output = torch.from_numpy(output)
        if target.shape[0] == 32: # last batch might be smaller than 32 (quick fix)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(data_loader.dataset)
    return 100. * correct / len(data_loader.dataset)

def test_tensorrt(model_name, data_loader):
    with open(model_name, "rb") as f:
        serialized_engine = f.read()
    runtime = trt.Runtime(logger)
    engine = runtime.deserialize_cuda_engine(serialized_engine)
    context = engine.create_execution_context()
    input_size = trt.volume(engine.get_binding_shape(0))
    output_size = trt.volume(engine.get_binding_shape(1))
    # Allocate device memory
    d_input = cuda.mem_alloc(input_size * 4)  # Assuming 4-byte float32 data type
    d_output = cuda.mem_alloc(output_size * 4)
    bindings=[int(d_input), int(d_output)]
    stream = cuda.Stream()
    h_output = np.empty(output_size, dtype=np.float32)
    test_loss = 0
    correct = 0
    for data, target in data_loader:
        # Create numpy arrays to hold input and output data
        h_input = data.numpy().astype(np.float32)
        # Transfer input data to device
        cuda.memcpy_htod_async(d_input, h_input, stream)
        # Execute model
        context.execute_async_v2(bindings, stream.handle, None)
        # Transfer predictions back
        cuda.memcpy_dtoh_async(h_output, d_output, stream)
        # Syncronize threads
        stream.synchronize()
        output = h_output.reshape(context.get_tensor_shape('output'))
        output = torch.from_numpy(output)
        if target.shape[0] == 32: # last batch might be smaller than 32 (quick fix)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(data_loader.dataset)
    del context
    del engine
    cuda.Context.pop()
    return 100. * correct / len(data_loader.dataset)

acc = test_onnx(onnx_path, test_loader)
print(f"Accuracy of the onnx model is {acc}%")

trtr_acc = test_tensorrt(trt_path, test_loader)
print(f"Accuracy of the tensorrt model is {trtr_acc}%")

Accuracy of the onnx model is 96.2%


<ipython-input-5-e7f70e092b44>:27: DeprecationWarning: Use get_tensor_shape instead.
  input_size = trt.volume(engine.get_binding_shape(0))
<ipython-input-5-e7f70e092b44>:28: DeprecationWarning: Use get_tensor_shape instead.
  output_size = trt.volume(engine.get_binding_shape(1))


Accuracy of the tensorrt model is 96.2%
